In [5]:
import sys
import xarray as xr
import numpy as np
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from dateutil.relativedelta import relativedelta
import bz2
import ftputil
import os

import glob
from datetime import datetime, timedelta
import pygrib
import pandas as pd

In [6]:
levels = [ 100.,  150.,  200.,  250.,  300.,  400.,  500.,  600.,  700., 800.,  850.,  900.,  925.,  950., 1000.]
gribnames = glob.glob("/home/opn/script/siam-netcdf-generator/coba/tes/envyosa/ecmwf/output/*.bin")

In [68]:

grbs = pygrib.open(gribnames[0])
grb = grbs.select(typeOfLevel='isobaricInhPa',shortName=['u'])
itime = grb[0].analDate 
lats, lons = grb[0].latlons()
lats, lons = lats[:,0], lons[0,:]
timestep = pd.date_range(start=itime + timedelta(hours=1), periods=90, freq='H')
levels = [ 100.,  150.,  200.,  250.,  300.,  400.,  500.,  600.,  700., 800.,  850.,  900.,  925.,  950., 1000.]
ds4 = xr.Dataset(coords=dict(
        time=(["time"],timestep),
        level=(["level"], levels),
        lat=(["lat"], lats),
        lon=(["lon"], lons)))


array(['d', 'gh', 'pv', 'q', 'r', 't', 'u', 'v', 'w', 'z'], dtype='<U2')

In [70]:
def sortFunc(e):
    return e['level'], e['shortName']
grb = grbs.select(typeOfLevel='isobaricInhPa',shortName=['r','t'])
grb = grb.sort(key=sortFunc)

now = datetime.now()
for grm in grb:
    varname = grm.shortName
    onlev = float(grm.level)      
    ds4[varname]=(('time','level','lon','lat'),np.empty((len(timestep),len(levels),len(lons),len(lats))))
    ds4[varname][:,:,:,:] = np.nan
    data = grm.values
    data = np.reshape(data.T,(len(lons),len(lats))) 
    ds4[varname].loc[dict(time=timestep[0], level=onlev)] = data
    print(datetime.now() - now)

TypeError: 'NoneType' object is not iterable

In [86]:
grbs.seek(0)
grbs = pygrib.open(gribnames[0])
grb = grbs.select(typeOfLevel='isobaricInhPa',shortName=['rh','t'])
grb.sort(key=sortFunc)

In [87]:
grb.sort(key=sortFunc)

In [108]:
grb

[109:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 100:fcst time 1 hrs:from 202402051200,
 86:Temperature:K (instant):regular_ll:isobaricInhPa:level 100:fcst time 1 hrs:from 202402051200,
 107:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 150:fcst time 1 hrs:from 202402051200,
 108:Temperature:K (instant):regular_ll:isobaricInhPa:level 150:fcst time 1 hrs:from 202402051200,
 87:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 200:fcst time 1 hrs:from 202402051200,
 74:Temperature:K (instant):regular_ll:isobaricInhPa:level 200:fcst time 1 hrs:from 202402051200,
 95:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 250:fcst time 1 hrs:from 202402051200,
 84:Temperature:K (instant):regular_ll:isobaricInhPa:level 250:fcst time 1 hrs:from 202402051200,
 70:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 300:fcst time 1 hrs:from 202402051200,
 63:Temperature:K (instant):regular_ll:isobaricInhPa:level 300:fcst time 1 hrs:from 2

In [118]:
r = []
t = []
j=1
i=0
while i < 30:
    print(i)
    r.append(grb[i].values)
    t.append(grb[j].values)
    i+=2
    j+=2
t = np.array(t) - 273.15

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28


In [124]:
dumdum = np.empty((len(timestep),len(levels),len(lats),len(lons)))
dumdum[:,:,:,:] = np.nan
var4 = ['r','t','d', 'gh', 'pv', 'q', 'u', 'v', 'w', 'z']
for var in var4:
    ds4[var] =(('time','level','lat','lon'),dumdum)

def rtextract(grms):
    r = []
    t = []
    i=0
    while i < 30:
        r.append(grms[i].values)
        t.append(grms[i+1].values)
        i+=2
    t = np.array(t) - 273.15
    return r, t


ds4['r'][0,:,:,:] = r
ds4['t'][0,:,:,:] = t 

In [125]:
grbs = pygrib.open(gribnames[0])
grbs.seek(0)
grbres = grbs.select(typeOfLevel='isobaricInhPa',shortName=['d', 'gh', 'pv', 'q', 'u', 'v', 'w', 'z'])
grbres.sort(key=sortFunc)

In [148]:
def resextract(grms):
    d,gh,pv,q,u,v,w,z = ([] for i in range(8))
    i = 0
    j = 0
    nans = np.empty((np.shape(grms[0].values)))
    nans[:] = np.nan
    while i < 88:
        
        if j == 7 or j == 9 or j == 11 or j == 13:
            d.append(nans)
            gh.append(nans)
            pv.append(nans)
            q.append(nans)
            u.append(nans)
            v.append(nans)
            w.append(nans)
            z.append(nans)
            j+=1

        d.append(grms[i].values)
        gh.append(grms[i+1].values)
        pv.append(grms[i+2].values)
        q.append(grms[i+3].values)
        u.append(grms[i+4].values)
        v.append(grms[i+5].values)
        w.append(grms[i+6].values)
        z.append(grms[i+7].values)
        i+=8
        j+=1

    return d,gh,pv,q,u,v,w,z
        

In [149]:
d,gh,pv,q,u,v,w,z = resextract(grbres)

In [166]:
tes = [d,gh,pv,q,u,v,w,z]

In [168]:
np.shape(tes)

(8, 15, 521, 801)

In [105]:
'd', 'gh', 'pv', 'q', 'u', 'v', 'w', 'z'

(15, 521, 801)

In [169]:
ds32 = xr.open_mfdataset(gribnames, engine="cfgrib",
        backend_kwargs=dict(filter_by_keys={"typeOfLevel": "surface", 'edition': 2}),
        concat_dim='valid_time', combine='nested', parallel=True)

In [187]:
ds32 = ds32.drop_vars(['time','step'])

In [190]:
ds32 = ds32.drop_vars(['time','step','surface']).rename_dims({'valid_time':'time'}).rename_vars({'valid_time':'time'})

<xarray.Dataset>
Dimensions:    (latitude: 521, longitude: 801, time: 90)
Coordinates:
    surface    float64 0.0
  * latitude   (latitude) float64 40.0 39.88 39.75 39.62 ... -24.75 -24.88 -25.0
  * longitude  (longitude) float64 70.0 70.12 70.25 70.38 ... 169.8 169.9 170.0
  * time       (time) datetime64[ns] 2024-02-05T13:00:00 ... 2024-02-09T06:00:00
Data variables:
    kx         (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    ceil       (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-02-06T16:47 GRIB to CDM+CF via cfgrib-0.9.1...

In [191]:
ds3 = xr.open_mfdataset(gribnames, engine="cfgrib",
        backend_kwargs=dict(filter_by_keys={"typeOfLevel": "surface", 'edition': 1}),
        concat_dim='valid_time', combine='nested', parallel=True)

In [193]:
ds3 = ds3.drop_vars(['time','step','surface','number']).rename_dims({'valid_time':'time'}).rename_vars({'valid_time':'time'})

In [195]:
ds3

<xarray.Dataset>
Dimensions:    (latitude: 521, longitude: 801, time: 90)
Coordinates:
  * latitude   (latitude) float64 40.0 39.88 39.75 39.62 ... -24.75 -24.88 -25.0
  * longitude  (longitude) float64 70.0 70.12 70.25 70.38 ... 169.8 169.9 170.0
  * time       (time) datetime64[ns] 2024-02-05T13:00:00 ... 2024-02-09T06:00:00
Data variables:
    u100       (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    mcc        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    cp         (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    cape       (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    lcc        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    tcc        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    p3020      (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    hcc        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    sst        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    v100       (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    sp         (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    d2m        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    msl        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    gh         (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    z          (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    lsp        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    blh        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    fg10       (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    cbh        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    sro        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    deg0l      (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    cin        (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    fzra       (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
    sund       (time, latitude, longitude) float32 dask.array<chunksize=(1, 521, 801), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-02-06T17:00 GRIB to CDM+CF via cfgrib-0.9.1...

In [107]:
len(lats)

521

In [ ]:

# gribs reading and writing iteration (isobaric level)
grbsbulk = [pygrib.open(f) for f in gribnames]
grbsel = [f.select(typeOfLevel="isobaricInhPa") for f in grbsbulk]
for grbsel in enumerate(grbsel):
    now = datetime.now()
    for grm in grbsel[1]:
        varname = grm.shortName
        onlev = float(grm.level)      
        ds4[varname]=(('time','level','lon','lat'),np.empty((len(timestep),len(levels),len(lons),len(lats))))
        ds4[varname][:,:,:,:] = np.nan
        data = grm.values
        data = np.reshape(data.T,(len(lons),len(lats))) 
        ds4[varname].loc[dict(time=timestep[grbsel[0]], level=onlev)] = data
    print(datetime.now() - now)

In [2]:
import xarray as xr

In [3]:
ds = xr.open_dataset('/mnt/data/output/ECMWF_V2/2024/02/ecmwf_20240206_1200.nc')

In [4]:
ds

<xarray.Dataset>
Dimensions:  (latitude: 521, level: 15, longitude: 801, time: 90)
Coordinates:
  * time     (time) datetime64[ns] 2024-02-06T13:00:00 ... 2024-02-10T06:00:00
  * level    (level) float64 100.0 150.0 200.0 250.0 ... 900.0 925.0 950.0 1e+03
    lat      (latitude) float64 ...
    lon      (longitude) float64 ...
Dimensions without coordinates: latitude, longitude
Data variables:
    r        (time, level, latitude, longitude) float64 ...
    t        (time, level, latitude, longitude) float64 ...
    gh       (time, level, latitude, longitude) float64 ...
    pv       (time, level, latitude, longitude) float64 ...
    u        (time, level, latitude, longitude) float64 ...
    v        (time, level, latitude, longitude) float64 ...
    w        (time, level, latitude, longitude) float64 ...
    tcc      (time, latitude, longitude) float32 ...
    cape     (time, latitude, longitude) float32 ...
    p3020    (time, latitude, longitude) float32 ...
    mcc      (time, latitude, longitude) float32 ...
    cp       (time, latitude, longitude) float32 ...
    lcc      (time, latitude, longitude) float32 ...
    hcc      (time, latitude, longitude) float32 ...
    tp       (time, latitude, longitude) float32 ...
    sp       (time, latitude, longitude) float32 ...
    msl      (time, latitude, longitude) float32 ...
    d2m      (time, latitude, longitude) float32 ...
    u10      (time, latitude, longitude) float32 ...
    t2m      (time, latitude, longitude) float32 ...
    v10      (time, latitude, longitude) float32 ...
    lsp      (time, latitude, longitude) float32 ...
    blh      (time, latitude, longitude) float32 ...
    fg10     (time, latitude, longitude) float32 ...
    cbh      (time, latitude, longitude) float32 ...
    sro      (time, latitude, longitude) float32 ...
    deg0l    (time, latitude, longitude) float32 ...
    sund     (time, latitude, longitude) float32 ...
    kx       (time, latitude, longitude) float32 ...
    ceil     (time, latitude, longitude) float32 ...

In [25]:
ds.drop_vars(['u100','v100','fzra','cin','d','z','sst','q'])

<xarray.Dataset>
Dimensions:    (lat: 521, latitude: 521, level: 15, lon: 801, longitude: 801, time: 90)
Coordinates:
  * time       (time) datetime64[ns] 2024-02-06T01:00:00 ... 2024-02-09T18:00:00
  * level      (level) float64 100.0 150.0 200.0 250.0 ... 925.0 950.0 1e+03
  * lat        (lat) float64 40.0 39.88 39.75 39.62 ... -24.75 -24.88 -25.0
  * lon        (lon) float64 70.0 70.12 70.25 70.38 ... 169.6 169.8 169.9 170.0
  * latitude   (latitude) float64 40.0 39.88 39.75 39.62 ... -24.75 -24.88 -25.0
  * longitude  (longitude) float64 70.0 70.12 70.25 70.38 ... 169.8 169.9 170.0
Data variables:
    r          (time, level, lat, lon) float64 ...
    t          (time, level, lat, lon) float64 ...
    gh         (time, level, lat, lon) float64 ...
    pv         (time, level, lat, lon) float64 ...
    u          (time, level, lat, lon) float64 ...
    v          (time, level, lat, lon) float64 ...
    w          (time, level, lat, lon) float64 ...
    u10        (time, latitude, longitude) float32 ...
    hcc        (time, latitude, longitude) float32 ...
    tcc        (time, latitude, longitude) float32 ...
    cp         (time, latitude, longitude) float32 ...
    mcc        (time, latitude, longitude) float32 ...
    cape       (time, latitude, longitude) float32 ...
    p3020      (time, latitude, longitude) float32 ...
    lcc        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
    sp         (time, latitude, longitude) float32 ...
    d2m        (time, latitude, longitude) float32 ...
    msl        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    lsp        (time, latitude, longitude) float32 ...
    blh        (time, latitude, longitude) float32 ...
    fg10       (time, latitude, longitude) float32 ...
    cbh        (time, latitude, longitude) float32 ...
    sro        (time, latitude, longitude) float32 ...
    deg0l      (time, latitude, longitude) float32 ...
    sund       (time, latitude, longitude) float32 ...
    kx         (time, latitude, longitude) float32 ...
    ceil       (time, latitude, longitude) float32 ...

In [7]:
grbs = pygrib.open(gribnames[0])

In [21]:
grbs.seek(0)
name = [[f'{x.name},{x.shortName}'] for x in grbs]

In [22]:
name = np.unique(name)

In [ ]:
100 u,v ; fzra ; cin ; d ; z ; sst ; q
{'r':'rh'}

In [23]:
name

array(['0 degrees C isothermal level (atm),deg0l',
       '10 metre U wind component,10u', '10 metre V wind component,10v',
       '10 metre wind gust since previous post-processing,10fg',
       '100 metre U wind component,100u',
       '100 metre V wind component,100v',
       '2 metre dewpoint temperature,2d', '2 metre temperature,2t',
       'Accumulated freezing rain,fzra', 'Boundary layer height,blh',
       'Ceiling,ceil', 'Cloud base height,cbh',
       'Convective available potential energy,cape',
       'Convective inhibition,cin', 'Convective precipitation,cp',
       'Divergence,d', 'Geopotential height,gh', 'Geopotential,z',
       'High cloud cover,hcc',
       'Instantaneous total lightning flash density,litoti', 'K index,kx',
       'Large-scale precipitation,lsp', 'Low cloud cover,lcc',
       'Mean sea level pressure,msl', 'Medium cloud cover,mcc',
       'Potential vorticity,pv', 'Relative humidity,r',
       'Sea surface temperature,sst', 'Specific humidity,q',
    

In [26]:
dtn = datetime.now()

In [37]:
f'{dtn:%d}'

'07'